# Technical Analysis - Comprehensive Guide

This notebook demonstrates comprehensive technical analysis using various indicators.

## Learning Objectives:
- Calculate and interpret trend indicators (SMA, EMA, MACD, ADX)
- Calculate and interpret momentum indicators (RSI, Stochastic, Williams %R)
- Calculate and interpret volatility indicators (Bollinger Bands, ATR, Keltner Channels)
- Calculate and interpret volume indicators (OBV, VWAP, A/D Line)
- Detect trading signals from technical indicators
- Visualize indicators on price charts
- Combine multiple indicators for analysis

## Table of Contents:
1. [Setup and Data](#setup)
2. [Trend Indicators](#trend)
3. [Momentum Indicators](#momentum)
4. [Volatility Indicators](#volatility)
5. [Volume Indicators](#volume)
6. [Signal Detection](#signals)
7. [Multi-Indicator Analysis](#multi)
8. [Multi-Stock Comparison](#comparison)

<a id='setup'></a>
## 1. Setup and Data Loading

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src.data.fetcher import get_stock_data
from src.indicators import (
    TrendIndicators, MomentumIndicators, 
    VolatilityIndicators, VolumeIndicators
)
from src.visualization.charts import StockCharts

# Configure display
pd.set_option('display.max_columns', None)
%matplotlib inline

print("Imports successful!")

In [ ]:
# Fetch stock data - using Apple as example
ticker = 'AAPL'
df = get_stock_data(ticker, start='2023-01-01')

print(f"Data loaded for {ticker}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(f"Shape: {df.shape}")
display(df.tail())

<a id='trend'></a>
## 2. Trend Indicators

Trend indicators help identify the direction and strength of price movements.

### 2.1 Simple Moving Averages (SMA)

In [ ]:
# Calculate multiple SMAs
df['SMA_20'] = TrendIndicators.sma(df, period=20)
df['SMA_50'] = TrendIndicators.sma(df, period=50)
df['SMA_200'] = TrendIndicators.sma(df, period=200)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2)
plt.plot(df.index, df['SMA_20'], label='SMA 20', alpha=0.7)
plt.plot(df.index, df['SMA_50'], label='SMA 50', alpha=0.7)
plt.plot(df.index, df['SMA_200'], label='SMA 200', alpha=0.7)
plt.title(f'{ticker} - Moving Averages', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nMoving Averages:")
print(df[['Close', 'SMA_20', 'SMA_50', 'SMA_200']].tail())

### 2.2 Exponential Moving Averages (EMA)

In [ ]:
# Calculate EMAs
df['EMA_12'] = TrendIndicators.ema(df, period=12)
df['EMA_26'] = TrendIndicators.ema(df, period=26)

# Compare SMA vs EMA
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2, alpha=0.5)
plt.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=2)
plt.plot(df.index, df['EMA_12'], label='EMA 12', linewidth=2, linestyle='--')
plt.title(f'{ticker} - SMA vs EMA', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("EMA responds faster to price changes than SMA")

### 2.3 MACD (Moving Average Convergence Divergence)

In [ ]:
# Calculate MACD
macd_df = TrendIndicators.macd(df, fast=12, slow=26, signal=9)
df = pd.concat([df, macd_df], axis=1)

# Get column names
macd_col = [col for col in df.columns if 'MACD_' in col and 'MACDs' not in col and 'MACDh' not in col][0]
signal_col = [col for col in df.columns if 'MACDs' in col][0]
hist_col = [col for col in df.columns if 'MACDh' in col][0]

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Price and EMAs
ax1.plot(df.index, df['Close'], label='Close Price', linewidth=2)
ax1.plot(df.index, df['EMA_12'], label='EMA 12', alpha=0.7)
ax1.plot(df.index, df['EMA_26'], label='EMA 26', alpha=0.7)
ax1.set_title(f'{ticker} - Price and MACD', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MACD
ax2.plot(df.index, df[macd_col], label='MACD Line', linewidth=2)
ax2.plot(df.index, df[signal_col], label='Signal Line', linewidth=2)
ax2.bar(df.index, df[hist_col], label='Histogram', alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax2.set_ylabel('MACD')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMACD Interpretation:")
print("- MACD > Signal Line = Bullish")
print("- MACD < Signal Line = Bearish")
print("- Histogram shows the difference between MACD and Signal")

### 2.4 Golden Cross and Death Cross Detection

In [ ]:
# Detect crosses
golden_cross = TrendIndicators.detect_golden_cross(df, fast=50, slow=200)
death_cross = TrendIndicators.detect_death_cross(df, fast=50, slow=200)

print(f"Golden Crosses (SMA50 crosses above SMA200): {golden_cross.sum()}")
print(f"Death Crosses (SMA50 crosses below SMA200): {death_cross.sum()}")

# Show dates
if golden_cross.sum() > 0:
    print(f"\nGolden Cross dates:")
    print(df[golden_cross].index.tolist())
    
if death_cross.sum() > 0:
    print(f"\nDeath Cross dates:")
    print(df[death_cross].index.tolist())

<a id='momentum'></a>
## 3. Momentum Indicators

Momentum indicators measure the rate of change and help identify overbought/oversold conditions.

### 3.1 RSI (Relative Strength Index)

In [ ]:
# Calculate RSI
df['RSI'] = MomentumIndicators.rsi(df, period=14)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_title(f'{ticker} - Price and RSI', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# RSI
ax2.plot(df.index, df['RSI'], linewidth=2, color='purple')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, label='Overbought (70)')
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, label='Oversold (30)')
ax2.axhline(y=50, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.fill_between(df.index, 70, 100, alpha=0.1, color='red')
ax2.fill_between(df.index, 0, 30, alpha=0.1, color='green')
ax2.set_ylabel('RSI')
ax2.set_xlabel('Date')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRSI Statistics:")
print(f"Current RSI: {df['RSI'].iloc[-1]:.2f}")
print(f"Average RSI: {df['RSI'].mean():.2f}")
print(f"Times above 70 (overbought): {(df['RSI'] > 70).sum()}")
print(f"Times below 30 (oversold): {(df['RSI'] < 30).sum()}")

### 3.2 Stochastic Oscillator

In [ ]:
# Calculate Stochastic
stoch = MomentumIndicators.stochastic(df, k_period=14, d_period=3)
df = pd.concat([df, stoch], axis=1)

# Get column names
k_col = [col for col in df.columns if 'STOCHk' in col][0]
d_col = [col for col in df.columns if 'STOCHd' in col][0]

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_title(f'{ticker} - Price and Stochastic', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# Stochastic
ax2.plot(df.index, df[k_col], label='%K', linewidth=2)
ax2.plot(df.index, df[d_col], label='%D', linewidth=2)
ax2.axhline(y=80, color='red', linestyle='--', linewidth=1, label='Overbought (80)')
ax2.axhline(y=20, color='green', linestyle='--', linewidth=1, label='Oversold (20)')
ax2.fill_between(df.index, 80, 100, alpha=0.1, color='red')
ax2.fill_between(df.index, 0, 20, alpha=0.1, color='green')
ax2.set_ylabel('Stochastic')
ax2.set_xlabel('Date')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nStochastic Interpretation:")
print("- %K > 80: Overbought")
print("- %K < 20: Oversold")
print("- %K crosses above %D: Buy signal")
print("- %K crosses below %D: Sell signal")

### 3.3 Williams %R

In [ ]:
# Calculate Williams %R
df['Williams_R'] = MomentumIndicators.williams_r(df, period=14)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_title(f'{ticker} - Price and Williams %R', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# Williams %R
ax2.plot(df.index, df['Williams_R'], linewidth=2, color='orange')
ax2.axhline(y=-20, color='red', linestyle='--', linewidth=1, label='Overbought (-20)')
ax2.axhline(y=-80, color='green', linestyle='--', linewidth=1, label='Oversold (-80)')
ax2.fill_between(df.index, -20, 0, alpha=0.1, color='red')
ax2.fill_between(df.index, -100, -80, alpha=0.1, color='green')
ax2.set_ylabel('Williams %R')
ax2.set_xlabel('Date')
ax2.set_ylim(-100, 0)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Current Williams %R: {df['Williams_R'].iloc[-1]:.2f}")

<a id='volatility'></a>
## 4. Volatility Indicators

Volatility indicators measure price fluctuations and help identify breakout opportunities.

### 4.1 Bollinger Bands

In [ ]:
# Calculate Bollinger Bands
bb = VolatilityIndicators.bollinger_bands(df, period=20, std_dev=2.0)
df = pd.concat([df, bb], axis=1)

# Get column names
bbu_col = [col for col in df.columns if 'BBU_' in col][0]
bbm_col = [col for col in df.columns if 'BBM_' in col][0]
bbl_col = [col for col in df.columns if 'BBL_' in col][0]

# Plot
plt.figure(figsize=(14, 8))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2, color='blue')
plt.plot(df.index, df[bbu_col], label='Upper Band', linewidth=1.5, color='red', alpha=0.7)
plt.plot(df.index, df[bbm_col], label='Middle Band (SMA 20)', linewidth=1.5, color='gray', linestyle='--')
plt.plot(df.index, df[bbl_col], label='Lower Band', linewidth=1.5, color='green', alpha=0.7)
plt.fill_between(df.index, df[bbl_col], df[bbu_col], alpha=0.1)

plt.title(f'{ticker} - Bollinger Bands', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nBollinger Bands Interpretation:")
print("- Price touching upper band: Potentially overbought")
print("- Price touching lower band: Potentially oversold")
print("- Narrow bands: Low volatility (potential breakout coming)")
print("- Wide bands: High volatility")

### 4.2 Average True Range (ATR)

In [ ]:
# Calculate ATR
df['ATR'] = VolatilityIndicators.atr(df, period=14)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_title(f'{ticker} - Price and ATR', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# ATR
ax2.plot(df.index, df['ATR'], linewidth=2, color='purple')
ax2.fill_between(df.index, 0, df['ATR'], alpha=0.3)
ax2.set_ylabel('ATR')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nATR Statistics:")
print(f"Current ATR: ${df['ATR'].iloc[-1]:.2f}")
print(f"Average ATR: ${df['ATR'].mean():.2f}")
print(f"Max ATR: ${df['ATR'].max():.2f}")
print("\nATR measures volatility - higher values mean more volatile prices")

### 4.3 Keltner Channels

In [ ]:
# Calculate Keltner Channels
kc = VolatilityIndicators.keltner_channels(df, period=20, atr_period=10, multiplier=2.0)
df = pd.concat([df, kc], axis=1)

# Get column names
kcu_col = [col for col in df.columns if 'KCU_' in col][0]
kcm_col = [col for col in df.columns if 'KCM_' in col][0]
kcl_col = [col for col in df.columns if 'KCL_' in col][0]

# Plot
plt.figure(figsize=(14, 8))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2)
plt.plot(df.index, df[kcu_col], label='Upper Channel', linewidth=1.5, alpha=0.7)
plt.plot(df.index, df[kcm_col], label='Middle (EMA)', linewidth=1.5, linestyle='--', alpha=0.7)
plt.plot(df.index, df[kcl_col], label='Lower Channel', linewidth=1.5, alpha=0.7)
plt.fill_between(df.index, df[kcl_col], df[kcu_col], alpha=0.1)

plt.title(f'{ticker} - Keltner Channels', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Keltner Channels use ATR instead of standard deviation (unlike Bollinger Bands)")

<a id='volume'></a>
## 5. Volume Indicators

Volume indicators help confirm price movements and identify divergences.

### 5.1 On-Balance Volume (OBV)

In [ ]:
# Calculate OBV
df['OBV'] = VolumeIndicators.obv(df)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [1, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2, color='blue')
ax1.set_title(f'{ticker} - Price and OBV', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# OBV
ax2.plot(df.index, df['OBV'], linewidth=2, color='orange')
ax2.set_ylabel('OBV')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nOBV Interpretation:")
print("- OBV rising with price: Strong uptrend (volume confirms)")
print("- OBV falling with price: Strong downtrend (volume confirms)")
print("- Divergence: OBV and price move in opposite directions (warning signal)")

### 5.2 Volume Weighted Average Price (VWAP)

In [ ]:
# Calculate VWAP
df['VWAP'] = VolumeIndicators.vwap(df)

# Plot
plt.figure(figsize=(14, 8))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2)
plt.plot(df.index, df['VWAP'], label='VWAP', linewidth=2, linestyle='--', alpha=0.7)
plt.title(f'{ticker} - Price and VWAP', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nVWAP Interpretation:")
print("- Price above VWAP: Generally bullish")
print("- Price below VWAP: Generally bearish")
print("- Often used by institutions as a benchmark")

### 5.3 Accumulation/Distribution Line

In [ ]:
# Calculate A/D Line
df['AD_Line'] = VolumeIndicators.ad_line(df)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [1, 1]})

# Price
ax1.plot(df.index, df['Close'], linewidth=2)
ax1.set_title(f'{ticker} - Price and A/D Line', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.grid(True, alpha=0.3)

# A/D Line
ax2.plot(df.index, df['AD_Line'], linewidth=2, color='green')
ax2.set_ylabel('A/D Line')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("A/D Line shows accumulation (buying pressure) vs distribution (selling pressure)")

### 5.4 Volume Analysis

In [ ]:
# Detect volume spikes and unusual volume
volume_spikes = VolumeIndicators.detect_volume_spike(df, threshold=2.0, period=20)
unusual_volume = VolumeIndicators.detect_unusual_volume(df, std_dev=2.0, period=20)

print(f"Volume spikes (>2x average): {volume_spikes.sum()}")
print(f"Unusual volume periods (>2 std dev): {unusual_volume.sum()}")

# Show recent spikes
if volume_spikes.sum() > 0:
    recent_spikes = df[volume_spikes].tail(5)
    print("\nRecent volume spike dates and volumes:")
    print(recent_spikes[['Close', 'Volume']])

<a id='signals'></a>
## 6. Signal Detection

Detect actionable trading signals from indicators.

In [ ]:
# RSI signals
rsi_oversold, rsi_overbought = MomentumIndicators.detect_rsi_signals(df['RSI'])

# Stochastic signals
stoch_buy, stoch_sell = MomentumIndicators.detect_stochastic_signals(stoch)

# Bollinger Band signals
bb_upper_break, bb_lower_break = VolatilityIndicators.detect_bollinger_breakout(df, bb)

print("=== SIGNAL SUMMARY ===")
print(f"\nRSI Signals:")
print(f"  Oversold (<30): {rsi_oversold.sum()} times")
print(f"  Overbought (>70): {rsi_overbought.sum()} times")

print(f"\nStochastic Signals:")
print(f"  Buy signals: {stoch_buy.sum()}")
print(f"  Sell signals: {stoch_sell.sum()}")

print(f"\nBollinger Band Breakouts:")
print(f"  Upper band breaks: {bb_upper_break.sum()}")
print(f"  Lower band breaks: {bb_lower_break.sum()}")

print(f"\nMoving Average Crosses:")
print(f"  Golden crosses: {golden_cross.sum()}")
print(f"  Death crosses: {death_cross.sum()}")

<a id='multi'></a>
## 7. Multi-Indicator Analysis

Combine multiple indicators for more robust analysis.

In [ ]:
# Create a comprehensive view
latest = df.iloc[-1]

print("=== CURRENT MARKET ANALYSIS ===")
print(f"\nTicker: {ticker}")
print(f"Date: {df.index[-1].date()}")
print(f"Close Price: ${latest['Close']:.2f}")

print(f"\nTrend Analysis:")
print(f"  SMA 20: ${latest['SMA_20']:.2f} - Price is {'ABOVE' if latest['Close'] > latest['SMA_20'] else 'BELOW'}")
print(f"  SMA 50: ${latest['SMA_50']:.2f} - Price is {'ABOVE' if latest['Close'] > latest['SMA_50'] else 'BELOW'}")
print(f"  SMA 200: ${latest['SMA_200']:.2f} - Price is {'ABOVE' if latest['Close'] > latest['SMA_200'] else 'BELOW'}")
print(f"  MACD: {'BULLISH' if latest[macd_col] > latest[signal_col] else 'BEARISH'}")

print(f"\nMomentum Analysis:")
print(f"  RSI: {latest['RSI']:.2f} - ", end='')
if latest['RSI'] > 70:
    print("OVERBOUGHT")
elif latest['RSI'] < 30:
    print("OVERSOLD")
else:
    print("NEUTRAL")
print(f"  Stochastic %K: {latest[k_col]:.2f}")
print(f"  Williams %R: {latest['Williams_R']:.2f}")

print(f"\nVolatility Analysis:")
print(f"  ATR: ${latest['ATR']:.2f}")
print(f"  Bollinger Band Position: ", end='')
bb_pct = (latest['Close'] - latest[bbl_col]) / (latest[bbu_col] - latest[bbl_col])
print(f"{bb_pct*100:.1f}% (0%=lower, 100%=upper)")

print(f"\nVolume Analysis:")
print(f"  Current Volume: {latest['Volume']:,.0f}")
print(f"  Price vs VWAP: {'ABOVE' if latest['Close'] > latest['VWAP'] else 'BELOW'}")

# Overall assessment
bullish_signals = 0
bearish_signals = 0

if latest['Close'] > latest['SMA_50']: bullish_signals += 1
else: bearish_signals += 1

if latest[macd_col] > latest[signal_col]: bullish_signals += 1
else: bearish_signals += 1

if latest['RSI'] > 50: bullish_signals += 1
elif latest['RSI'] < 50: bearish_signals += 1

if latest['Close'] > latest['VWAP']: bullish_signals += 1
else: bearish_signals += 1

print(f"\n=== OVERALL ASSESSMENT ===")
print(f"Bullish signals: {bullish_signals}")
print(f"Bearish signals: {bearish_signals}")
print(f"Market bias: {'BULLISH' if bullish_signals > bearish_signals else 'BEARISH' if bearish_signals > bullish_signals else 'NEUTRAL'}")

<a id='comparison'></a>
## 8. Multi-Stock Technical Comparison

Compare technical indicators across multiple stocks.

In [ ]:
# Fetch multiple stocks
tickers = ['AAPL', 'MSFT', 'GOOGL']
stocks = {}

for t in tickers:
    df_stock = get_stock_data(t, start='2023-01-01')
    df_stock['RSI'] = MomentumIndicators.rsi(df_stock)
    stocks[t] = df_stock

print("Loaded data for:", tickers)

In [ ]:
# Compare RSI
plt.figure(figsize=(14, 6))
for ticker, df_stock in stocks.items():
    plt.plot(df_stock.index, df_stock['RSI'], label=ticker, linewidth=2)

plt.axhline(y=70, color='red', linestyle='--', linewidth=1, alpha=0.5)
plt.axhline(y=30, color='green', linestyle='--', linewidth=1, alpha=0.5)
plt.title('RSI Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.ylim(0, 100)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Technical summary for all stocks
summary_data = []

for ticker, df_stock in stocks.items():
    latest = df_stock.iloc[-1]
    summary_data.append({
        'Ticker': ticker,
        'Close': f"${latest['Close']:.2f}",
        'RSI': f"{latest['RSI']:.2f}",
        'Volume': f"{latest['Volume']:,.0f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\nTechnical Summary:")
display(summary_df)

## Summary

In this notebook, we covered:

### Trend Indicators
- Simple and Exponential Moving Averages
- MACD for trend changes
- Golden/Death cross detection

### Momentum Indicators
- RSI for overbought/oversold conditions
- Stochastic Oscillator for momentum shifts
- Williams %R for market timing

### Volatility Indicators
- Bollinger Bands for volatility and breakouts
- ATR for measuring volatility
- Keltner Channels for trend confirmation

### Volume Indicators
- OBV for volume trend confirmation
- VWAP for intraday levels
- A/D Line for accumulation/distribution

### Key Takeaways
1. No single indicator is perfect - use multiple indicators
2. Look for confirmation across different indicator types
3. Volume should confirm price movements
4. Divergences between price and indicators can signal reversals
5. Adjust indicator parameters based on your trading timeframe

## Next Steps

In the next notebook (`03_fundamental_analysis.ipynb`), we'll learn:
- Financial ratio analysis
- Company valuation methods
- Earnings and growth metrics
- Peer comparison